In [1]:
import pandas as pd
test_data = pd.read_csv(
        f"../files/input/test_data.csv.zip",
        index_col=False,
        compression="zip",
    )
print(test_data.head())

   ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   2     120000    2          2         2   26     -1      2      0      0   
1  10      20000    1          3         2   35     -2     -2     -2     -2   
2  11     200000    2          3         2   34      0      0      2      0   
3  15     250000    1          1         2   29      0      0      0      0   
4  16      50000    2          3         3   23      1      2      0      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...       3272       3455       3261         0      1000      1000   
1  ...          0      13007      13912         0         0         0   
2  ...       2513       1828       3731      2306        12        50   
3  ...      59696      56875      55512      3000      3000      3000   
4  ...      28771      29531      30211         0      1500      1100   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default payment next month  
0      1000         0   

In [2]:
train_data = pd.read_csv(
        f"../files/input/train_data.csv.zip",
        index_col=False,
        compression="zip",
    )
print(test_data.head())

   ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   2     120000    2          2         2   26     -1      2      0      0   
1  10      20000    1          3         2   35     -2     -2     -2     -2   
2  11     200000    2          3         2   34      0      0      2      0   
3  15     250000    1          1         2   29      0      0      0      0   
4  16      50000    2          3         3   23      1      2      0      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...       3272       3455       3261         0      1000      1000   
1  ...          0      13007      13912         0         0         0   
2  ...       2513       1828       3731      2306        12        50   
3  ...      59696      56875      55512      3000      3000      3000   
4  ...      28771      29531      30211         0      1500      1100   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default payment next month  
0      1000         0   

In [ ]:
test_data=test_data.drop(columns="ID")

In [ ]:
train_data=train_data.drop(columns="ID")

In [ ]:
test_data.columns.values[-1]='default'

In [ ]:
test_data.columns

In [ ]:
train_data.columns.values[-1]='default'

In [ ]:
test_data=test_data.dropna()
train_data=train_data.dropna()

In [ ]:
train_data["EDUCATION"].value_counts()

In [3]:
def clean_data(df):
    # 1. Eliminar la columna 'ID' si existe
    if 'ID' in df.columns:
        df = df.drop(columns=['ID'])
    # 2. Renombrar la columna a 'default' si existe
    if "default payment next month" in df.columns:
        df = df.rename(columns={'default payment next month': 'default'})
    # 3. Reemplazar valores >4 
    if 'EDUCATION' in df.columns:
        df['EDUCATION']=df['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
    # 4. Eliminar filas con valores NaN
    df = df.dropna()
    
    return df

In [4]:
train_data=clean_data(train_data)
test_data=clean_data(test_data)

In [5]:
train_data['EDUCATION'].value_counts()

EDUCATION
2    9759
1    7480
3    3427
4     325
0       9
Name: count, dtype: int64

In [9]:
x_train = train_data.drop(columns=['default'])
y_train = train_data['default']

x_test = test_data.drop(columns=['default'])
y_test = test_data['default']


In [16]:
def make_pipeline(estimator):

    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import OneHotEncoder

    cols=['SEX','EDUCATION','MARRIAGE','AGE']
    transformer = ColumnTransformer(
        transformers=[
            ("ohe", OneHotEncoder(dtype="int",handle_unknown="ignore"),cols),
        ],
        remainder="passthrough",
    )

    pipeline = Pipeline(
        steps=[
            ("tranformer", transformer),
            ("estimator", estimator),
        ],
        verbose=False,
    )

    return pipeline

In [21]:
# Modelo estimador
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf_estimator=make_pipeline(rf)


In [34]:
## Paso 4 Optimizar hiperparámetros 10 splits
from sklearn.model_selection import GridSearchCV
# Grid mínimo para cumplir la especificación
param_grid = {"estimator__n_estimators": [100]}

# GridSearchCV con 10 folds (usa cross-validation interna)
rf_estimator = GridSearchCV(
    estimator=rf_estimator,
    param_grid=param_grid,
    scoring="balanced_accuracy",
    cv=10,
    n_jobs=-1
)


In [19]:
print("Balanced accuracy promedio:", scores)

Balanced accuracy promedio: [0.65589733 0.63412787 0.65312018 0.65456534 0.66356256 0.66213709
 0.65368043 0.62926942 0.65191061 0.65535214]


In [22]:
rf_estimator.fit(x_train,y_train)

c:\02_Predictiva\LAB-01-prediccion-del-default-usando-rf-kcastanoa\.venv\lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('tranformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(dtype='int',
                                                                handle_unknown='ignore'),
                                                  ['SEX', 'EDUCATION',
                                                   'MARRIAGE', 'AGE'])])),
                ('estimator', RandomForestClassifier(random_state=42))])

In [25]:
def save_estimator(estimator):
    import gzip
    import pickle
    import os
    # Crear carpeta destino si no existe
    os.makedirs("../files/models", exist_ok=True)
    # Guardar modelo comprimido en formato .pkl.gz
    with gzip.open("../files/models/model.pkl.gz", "wb") as file:
        pickle.dump(estimator, file)

In [26]:
save_estimator(rf_estimator)

In [29]:
import json
import os
import gzip
import pickle
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score

with gzip.open("../files/models/model.pkl.gz", "rb") as f:
    loaded_model = pickle.load(f)

y_train_pred = rf_estimator.predict(x_train)
y_test_pred = rf_estimator.predict(x_test)

metrics = [
    {
        "dataset": "train",
        "precision": precision_score(y_train, y_train_pred),
        "balanced_accuracy": balanced_accuracy_score(y_train, y_train_pred),
        "recall": recall_score(y_train, y_train_pred),
        "f1_score": f1_score(y_train, y_train_pred)
    },
    {
        "dataset": "test",
        "precision": precision_score(y_test, y_test_pred),
        "balanced_accuracy": balanced_accuracy_score(y_test, y_test_pred),
        "recall": recall_score(y_test, y_test_pred),
        "f1_score": f1_score(y_test, y_test_pred)
    }
]

os.makedirs("../files/output", exist_ok=True)

with open("../files/output/metrics.json", "w") as f:
    for row in metrics:
        f.write(json.dumps(row) + "\n")

In [30]:
print(metrics)

[{'dataset': 'train', 'precision': 0.9989417989417989, 'balanced_accuracy': np.float64(0.9991059439575205), 'recall': 0.9985191453353078, 'f1_score': 0.9987304274227676}, {'dataset': 'test', 'precision': 0.6445414847161572, 'balanced_accuracy': np.float64(0.6645965687845039), 'recall': 0.3865898376113148, 'f1_score': 0.48330058939096265}]


In [33]:
from sklearn.metrics import confusion_matrix
cm_train = confusion_matrix(y_train, y_train_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

os.makedirs("../files/output", exist_ok=True)

metrics = [
    {
        "type": "cm_matrix",
        "dataset": "train",
        "true_0": {"predicted_0": int(cm_train[0][0]), "predicted_1": int(cm_train[0][1])},
        "true_1": {"predicted_0": int(cm_train[1][0]), "predicted_1": int(cm_train[1][1])}
    },
    {
        "type": "cm_matrix",
        "dataset": "test",
        "true_0": {"predicted_0": int(cm_test[0][0]), "predicted_1": int(cm_test[0][1])},
        "true_1": {"predicted_0": int(cm_test[1][0]), "predicted_1": int(cm_test[1][1])}
    }
]

with open("../files/output/metrics.json", "a") as f:
    for row in metrics:
        f.write(json.dumps(row) + "\n")